In [ ]:
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import os
import json
from datetime import datetime
import multiprocessing as mp

In [ ]:
from models.cooling_phonons_v3 import CoolingPhonons
from models.graphene_optics import GrapheneTHG

In [ ]:
run_label = "fluence"
time_stamp = datetime.now().strftime("%Y-%m-%d-%H-%M-%S")
saveDir = "./data/02/%s_%s" % (time_stamp, run_label)
os.mkdir(saveDir)

In [ ]:
def oneRun(x):
    sysparams = {
        "tempK_eq": 300.0,
        "tau_ph": 1200.0,
        "pu_fluenceSI": x["pu_fluenceSI"],
        "pu_dt": 110.0,
        "pu_eph": 1.200,
        "pr_delay": 1000.0,
        "pr_fluenceSI": 20.0,
        "pr_dt": 150.0,
        "pr_eph": 0.320,
        "nsub": 1.45,
        "ntop": 1.0,
        "eF": 0.200
    }
    dyn = CoolingPhonons(**sysparams)
    runparams = {
        "tmax": (sysparams["pr_delay"] + sysparams["pr_dt"] * 2.0),
        "tnum": 101,
        "dt_approx": 10.0,
        "print_time": False    
    }
    dyn.run(**runparams)

    with open("%s/params-%02d.json" % (x["saveDir"], x["il"]), "w") as f:
        json.dump({"sysparams": sysparams, "runparams": runparams}, f, indent=4)

    np.savetxt("%s/dynamics-%02d.csv" % (x["saveDir"], x["il"]), dyn.dynamics_m, delimiter=",")
    np.savetxt("%s/thg-%02d.csv" % (x["saveDir"], x["il"]), np.c_[dyn.p["pr_tt"],dyn.p["etaTHG"]], delimiter=",")
    
    r = (x["pu_fluenceSI"], dyn.p["etaTHG_avg"], dyn.p["etaTHG_max"])
    return r

In [ ]:
pu_fluenceSI_list = np.linspace(20.0, 110.0, 10)
params_list = [{"il": il, "saveDir": saveDir, "pu_fluenceSI": pu_fluenceSI} for il,pu_fluenceSI in enumerate(pu_fluenceSI_list)]

with mp.Pool(10) as p:
    thg_list = p.map(oneRun, params_list)

thg_list = np.array(thg_list)
np.savetxt("%s/thg_vs_fluence.csv" % saveDir, thg_list, delimiter=",")

In [ ]:
mpl.style.use(["classic", "latex"])
fig = plt.figure(figsize=(3.5,3.0), frameon=True, dpi=150)
plt.axes([0.2, 0.2, 0.7, 0.7])
plt.plot(thg_list[:,0], thg_list[:,1], "-k")
plt.plot(thg_list[:,0], thg_list[:,2], "-r")
plt.xlim([0.0, 120.0])
plt.ylim([0.0, 1.5e-9])
plt.xlabel(r"${\cal F}_{\rm pu}~[\mu{\rm J}/{\rm cm}^{2}]$")
plt.ylabel(r"$\eta_{\rm THG}$")
fig.savefig("%s/plot_etaTHG_fluence.png" % saveDir, dpi=300)